In [1]:
import Pkg;
#Pkg.add("Flux");
#Pkg.add("ImageMagick");
#Pkg.update("Flux")
using Flux: onecold, onehotbatch, logitbinarycrossentropy, params, train!, DataLoader, Chain, Conv, Dense, maxpool, relu, softmax
using ImageMagick
using FileIO
using Images


In [2]:
function load_images(directory)
    image_paths = readdir(directory)
    image_paths = filter(x -> occursin(".jpg", x), image_paths)  # Filter PNG files
    image_paths = [joinpath(directory, img) for img in image_paths]
    images = [load(img) for img in image_paths]
    return images
end

load_images (generic function with 1 method)

In [3]:
good_images = load_images("C:\\Users\\Vincent Alexander\\OneDrive\\Desktop\\Julia\\processed_good_path")
bad_images = load_images("C:\\Users\\Vincent Alexander\\OneDrive\\Desktop\\Julia\\processed_defective_path")

good_labels = ones(Int, length(good_images))
bad_labels = zeros(Int, length(bad_images))

images = vcat(good_images, bad_images)
labels = vcat(good_labels, bad_labels)

1763-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [4]:
using Flux

model = Chain(
    Conv((3, 3), 1=>16, relu),
    MaxPool((2, 2)),
    Conv((3, 3), 16=>32, relu),
    MaxPool((2, 2)),
    x -> reshape(x, :, size(x, 4)),
    Dense(288, 2),
    sigmoid
)

Chain(
  Conv((3, 3), 1 => 16, relu),          # 160 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 16 => 32, relu),         # 4_640 parameters
  MaxPool((2, 2)),
  var"#17#18"(),
  Dense(288 => 2),                      # 578 parameters
  NNlib.σ,
)                   # Total: 6 arrays, 5_378 parameters, 22.086 KiB.

In [5]:
loss(x, y) = logitbinarycrossentropy(model(x), y)

loss (generic function with 1 method)

In [6]:
images = cat(images..., dims=4)
labels = onehotbatch(labels, 0:1)

2×1763 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [14]:
function calculate_accuracy(predictions, ground_truth)
    predicted_classes = onecold(predictions)
    ground_truth_classes = onecold(ground_truth)
    return sum(predicted_classes .== ground_truth_classes) / length(ground_truth_classes)
end

num_epochs = 10  
for epoch in 1:num_epochs
    for (X,y) in train_loader
        train!(loss, params(model), [(X,y)], ADAM(0.001))
    end
    
    if epoch % 1 == 0
        train_predictions = model(images)
        acc = calculate_accuracy(train_predictions, labels)
        println("Epoch: $epoch, Accuracy: $acc")
    end
end

MethodError: MethodError: no method matching Float32(::Array{Gray{Float32}, 3})

Closest candidates are:
  (::Type{T})(!Matched::VectorizationBase.Double{T}) where T<:Union{Float16, Float32, Float64, VectorizationBase.Vec{<:Any, <:Union{Float16, Float32, Float64}}, VectorizationBase.VecUnroll{var"#s45", var"#s44", var"#s43", V} where {var"#s45", var"#s44", var"#s43"<:Union{Float16, Float32, Float64}, V<:Union{Bool, Float16, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, SIMDTypes.Bit, VectorizationBase.AbstractSIMD{var"#s44", var"#s43"}}}}
   @ VectorizationBase C:\Users\Vincent Alexander\.julia\packages\VectorizationBase\0dXyA\src\special\double.jl:111
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number}
   @ Base char.jl:50
  (::Type{T})(!Matched::Base.TwicePrecision) where T<:Number
   @ Base twiceprecision.jl:266
  ...
